## Bankruptcy Prediction with LightGBM Classification

#### Read dataset

In [ ]:
dataset = spark.read.format("csv")\
  .option("header", True)\
  .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/company_bankruptcy_prediction_data.csv")
# print dataset size
print("records read: " + str(dataset.count()))

In [ ]:
# convert features to double type
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
for colName in dataset.columns:
  dataset = dataset.withColumn(colName, col(colName).cast(DoubleType()))
print("Schema: ")
dataset.printSchema()

In [ ]:
dataset.show(n=3, truncate=False, vertical=True)

#### Split the dataset into train and test

In [ ]:
train, test = dataset.randomSplit([0.85, 0.15], seed=1)

#### Add featurizer to convert features to vector

In [ ]:
from pyspark.ml.feature import VectorAssembler
feature_cols = dataset.columns[1:]
featurizer = VectorAssembler(
    inputCols=feature_cols,
    outputCol='features'
)
train_data = featurizer.transform(train)['Bankrupt?', 'features']
test_data = featurizer.transform(test)['Bankrupt?', 'features']

In [ ]:
train_data.show(10)

#### Check if the data is unbalanced

In [ ]:
train_data.groupBy("Bankrupt?").count().show()

#### Model Training

In [ ]:
from mmlspark.lightgbm import LightGBMClassifier
model = LightGBMClassifier(objective="binary", featuresCol="features", labelCol="Bankrupt?", isUnbalance=True)

In [ ]:
model = model.fit(train_data)

In [ ]:
from mmlspark.lightgbm import LightGBMClassificationModel
model.saveNativeModel("/lgbmcmodel")
model = LightGBMClassificationModel.loadNativeModelFromFile("/lgbmcmodel")

In [ ]:
print(model.getFeatureImportances())

#### Model Prediction

In [ ]:
predictions = model.transform(test_data)
predictions.limit(10).toPandas()

In [ ]:
ComputeModelStatistics(evaluationMetric="classification", labelCol="Bankrupt?", scoredLabelsCol="prediction").transform(predictions)

In [ ]:
from mmlspark.train import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric="classification", labelCol='Bankrupt?', scoredLabelsCol='prediction').transform(predictions)
display(metrics)